In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import wandb

In [4]:
def make_dataset(dir):
    print(os.path.isfile(dir))
    if os.path.isfile(dir):
        arr = np.genfromtxt(dir, dtype=str, encoding='utf-8')
        if arr.ndim:
            images = [i for i in arr]
        else:
            images = np.array([arr])
    return images

class PPG2ABPDataset_v3_base(Dataset):
    def __init__(self,data_flist,data_root = None,
                 data_len=1000, size=224, loader=None):
        self.data_root = data_root
        self.data_flist = data_flist
        self.flist = make_dataset(self.data_flist)
        # if data_len > 0:
        #     self.flist = flist[:int(data_len)]
        # else:
        #     self.flist = flist
        self.size = size
        self.data=self.load_npys()
        if data_len > 0:
            data_index = np.arange(0,len(self.data),max(len(self.data)//int(data_len),1)).astype(int)[:int(data_len)]
            self.data = self.data[data_index]
        else:
            self.data = self.data[:len(self.data)-len(self.data)%64]
        print("data prepared:" ,self.data.shape)
    def load_npys(self):
        data = []
        for f in self.flist:
            arr = np.load(self.data_root+"\\"+str(f))
            if len(arr) != 0:
                data.append(arr)
        data = np.concatenate(data)
        return data
    
    def __getitem__(self, index):
        ret = {}
        ret['gt_image'] = np.concatenate([self.data[index,:,0][np.newaxis, :].astype(np.float32).min(axis=-1),self.data[index,:,0][np.newaxis, :].astype(np.float32).max(axis=-1)],axis=-1)
        ret['cond_image'] = self.data[index,:,1][np.newaxis, :].astype(np.float32)
        ret['path'] = str(index)
        return ret

    def __len__(self):
        return self.data.shape[0]
    
class PPG2ABPDataset_v3_Train(PPG2ABPDataset_v3_base):
    def __init__(self, data_len=-1, size=224, loader=None, data_root=r"..\..\data\processed\BP_npy\1127_256_balanced\p00"):
        super().__init__(data_len=data_len,data_flist = r"..\data\processed\list\train_BP2.txt",data_root=data_root)

class PPG2ABPDataset_v3_Val(PPG2ABPDataset_v3_base):
    def __init__(self, data_len=1000, size=224, loader=None, data_root=r"..\..\data\processed\BP_npy\1127_256_balanced\p00"):
        super().__init__(data_len=data_len,data_flist = r"..\data\processed\list\val_BP2.txt",data_root=data_root)

class PPG2ABPDataset_v3_Test(PPG2ABPDataset_v3_base):
    def __init__(self, data_len=5000, size=224, loader=None, data_root=r"..\..\data\processed\BP_npy\1127_256_balanced\p00"):
        super().__init__(data_len=data_len,data_flist = r"..\data\processed\list\test_BP2.txt",data_root=data_root)         
    
class PPG2ABPDataset_v3_Predict(PPG2ABPDataset_v3_base):
    def __init__(self, data_len=-1,size=224, loader=None, data_root=r"..\..\data\processed\BP_npy\1127_256_balanced\p00"):
        super().__init__(data_flist = r"..\data\processed\list\predict_BP2.txt",data_root=data_root)


In [5]:

class EarlyStopping:
    """earlystoppingクラス"""

    def __init__(self, path, patience=5, verbose=False):
        """引数：最小値の非更新数カウンタ、表示設定、モデル格納path"""

        self.patience = patience    #設定ストップカウンタ
        self.verbose = verbose      #表示の有無
        self.counter = 0            #現在のカウンタ値
        self.best_score = None      #ベストスコア
        self.early_stop = False     #ストップフラグ
        # self.val_loss_min = np.Inf   #前回のベストスコア記憶用
        
        self.path = path             #ベストモデル格納path
        os.makedirs(os.path.dirname(self.path),exist_ok=True)
    def __call__(self, val_loss, model):
        """
        特殊(call)メソッド
        実際に学習ループ内で最小lossを更新したか否かを計算させる部分
        """
        score = val_loss

        if self.best_score is None:  #1Epoch目の処理
            self.best_score = score
            self.checkpoint(score, model)  #記録後にモデルを保存してスコア表示する
        elif score > self.best_score:  # ベストスコアを更新できなかった場合
            self.counter += 1   #ストップカウンタを+1
            if self.verbose:  #表示を有効にした場合は経過を表示
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')  #現在のカウンタを表示する
                print(f"the best of loss: {self.best_score:.5f}")
            if self.counter >= self.patience:  #設定カウントを上回ったらストップフラグをTrueに変更
                self.early_stop = True
        else:  #ベストスコアを更新した場合
            self.checkpoint(score, model)  #モデルを保存してスコア表示
            self.counter = 0  #ストップカウンタリセット

    def checkpoint(self, score, model):
        '''ベストスコア更新時に実行されるチェックポイント関数'''
        if self.verbose:  #表示を有効にした場合は、前回のベストスコアからどれだけ更新したか？を表示
            print(f'Validation loss decreased ({self.best_score:.5f} --> {score:.5f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)  #ベストモデルを指定したpathに保存
        self.best_score = score #その時のlossを記録する

In [34]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# from LPCL.FIR_layer_type2 import LinearPhaseFIRLayer_type2
# from LPCL.FIR_layer_type4 import LinearPhaseFIRLayer_type4

class CNN(nn.Module):
    def __init__(self, model_config, preprocess):
        super(CNN, self).__init__()
        
        self.preprocess = preprocess

        input_shape = model_config["input_shape"]
        self.input_channels = input_shape[0]
        
        # if preprocess == "LPCL":
        #     self.fir2_layers = nn.ModuleList([
        #         LinearPhaseFIRLayer_type2(filter_size=32, pad_edge="left_right"),
        #         LinearPhaseFIRLayer_type2(filter_size=64, pad_edge="left_right"),
        #         LinearPhaseFIRLayer_type2(filter_size=128, pad_edge="left_right"),
        #         LinearPhaseFIRLayer_type2(filter_size=256, pad_edge="left_right")
        #     ])

        #     self.fir4_layers = nn.ModuleList([
        #         LinearPhaseFIRLayer_type4(filter_size=32, pad_edge="left_right"),
        #         LinearPhaseFIRLayer_type4(filter_size=64, pad_edge="left_right"),
        #         LinearPhaseFIRLayer_type4(filter_size=128, pad_edge="left_right"),
        #         LinearPhaseFIRLayer_type4(filter_size=256, pad_edge="left_right")
        #     ])

        self.conv1 = nn.Conv1d(in_channels=self.input_channels, out_channels=8, kernel_size=4, padding=2)
        self.bn1 = nn.BatchNorm1d(8)
        
        self.conv2 = nn.Conv1d(in_channels=8, out_channels=16, kernel_size=4, padding=2)
        self.bn2 = nn.BatchNorm1d(16)
        
        self.conv3 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=4, padding=2)
        self.bn3 = nn.BatchNorm1d(32)
        
        self.conv4 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=4, padding=2)
        self.bn4 = nn.BatchNorm1d(64)
        
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(256, 1)  # Output layer for regression

    def forward(self, x):
        # if self.preprocess == "LPCL":
        #     fir2_outputs = [layer(x) for layer in self.fir2_layers]
        #     fir4_outputs = [layer(x) for layer in self.fir4_layers]
        #     x = torch.cat(fir2_outputs + fir4_outputs, dim=-1)

        x = F.relu(self.bn1(self.conv1(x)))
        
        x = F.max_pool1d(x, kernel_size=4)

        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool1d(x, kernel_size=4)

        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool1d(x, kernel_size=4)

        x = F.relu(self.bn4(self.conv4(x)))
        x = F.max_pool1d(x, kernel_size=4)

        x = self.flatten(x)
        # print("before dense",x.shape)
        x = self.fc(x)  # No activation for regression
        return x

def build_cnn(model_config, preprocess):
    model = CNN(model_config, preprocess)

    learning_rate = model_config["learning_rate"]
    optimizer_params = model_config["optimizer_params"]

    optimizer = optim.Adam(model.parameters(), lr=learning_rate, **optimizer_params)

    loss_fn = nn.MSELoss()  # Mean Squared Error Loss for regression
    metrics = model_config.get("metrics", [])  # Placeholder for evaluation

    return model, optimizer, loss_fn, metrics


In [7]:
def train_model(config):
    def create_figure(gt,pred):
      fig,ax = plt.subplots()
      ax.plot(gt,label="true")  
      ax.plot(gt,label="pred")
      ax.legend()
      return fig
    def log_img(gt,pred):
        gt, pred = gt[0].detach().clone().cpu().numpy(), pred[0].detach().clone().cpu().numpy()
        figure = create_figure(gt,pred)
        img = wandb.Image(figure)
        plt.clf()
        plt.close()
        return img
    # Initialize Weights & Biases (wandb)
    wandb.init(project="regression-training", config=config)
    config = wandb.config

    # Dataset and DataLoader
    train_dataset = PPG2ABPDataset_v3_Train(data_root=r"..\data\processed\BP_npy\250107_1152\p00")
    train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True)

    val_dataset = PPG2ABPDataset_v3_Val(data_len=-1,data_root=r"..\data\processed\BP_npy\250107_1152\p00")
    val_loader = DataLoader(val_dataset, batch_size=config.batch_size, shuffle=False)

    # Model, Loss, and Optimizer
    model = CNN(config, "not")
    criterion = nn.MSELoss()  # Mean Squared Error Loss for regression
    mae = nn.L1Loss()  # Mean Squared Error Loss for regression
    optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
    earlystopping = EarlyStopping(f"{config.output_path}/best.pth",config.patience,verbose=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    wandb.watch(model, log_freq=config.log_interval)
    for epoch in range(config.epochs):
        model.train()
        running_loss = 0.0
        running_loss_mae = 0.0
        # Training phase with progress bar
        train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{config.epochs} - Training", leave=False)
        for batch_idx,data in enumerate(train_loader_tqdm):
            gt = data["gt_image"]
            cond = data["cond_image"]
            gt,cond = gt.to(device), cond.to(device)
            # print(gt.shape,cond.shape)
            optimizer.zero_grad()
            outputs = model(cond)
            loss = criterion(outputs, gt)
            loss_mae = mae(outputs, gt)
            loss.backward()
            optimizer.step()
            # if batch_idx % config.log_interval == 0:
                # wandb.log({"loss": loss})
            # if batch_idx  == 0:
                # wandb.log({"train/loss": log_img(gt,outputs)})
            running_loss += loss.item()
            running_loss_mae += loss_mae.item()
            train_loader_tqdm.set_postfix(loss=loss.item())

        train_loss = running_loss / len(train_loader)
        train_loss_mae = running_loss_mae / len(train_loader)

        # Validation phase with progress bar
        model.eval()
        val_loss = 0.0
        val_loss_mae = 0.0
        val_loader_tqdm = tqdm(val_loader, desc=f"Epoch {epoch + 1}/{config.epochs} - Validation", leave=False)
        with torch.no_grad():
            for batch_idx,data in enumerate(val_loader_tqdm):
                gt = data["gt_image"]
                cond = data["cond_image"]
                gt,cond = gt.to(device), cond.to(device)
                outputs = model(cond)
                loss = criterion(outputs, gt)
                loss_mae = mae(outputs, gt)
                # if batch_idx % config.log_interval == 0:
                    # wandb.log({"loss": loss})
                # if batch_idx  == 0:
                    # wandb.log({"val/loss": log_img(gt,outputs)})
                val_loss += loss.item()
                val_loss_mae += loss_mae.item()

        val_loss /= len(val_loader)
        val_loss_mae /= len(val_loader)
        earlystopping(val_loss,model)
        if earlystopping.early_stop:
            print("Early Stopping!")
            break
        # Log metrics to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train/loss_epoch": train_loss,
            "train/mae": train_loss_mae,
            "val/loss_epoch": val_loss,
            "val/mae": val_loss_mae
        })

        print(f"Epoch [{epoch + 1}/{config.epochs}]"
              f" Train Loss: {train_loss:.4f}"
              f" Val Loss: {val_loss:.4f}")




In [8]:
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 24amj35 (bsa_mh). Use `wandb login --relogin` to force relogin


True

In [9]:
config = {
    "learning_rate": 1e-3,
    "batch_size": 32,
    "epochs":100,
    "log_interval":100,
    "input_shape":[1,1152],
    "output_path": "outputs\\cnn\\0108_2",
    "patience":10    
          }
train_model(config)

True
data prepared: (102144, 1152, 2)
True
data prepared: (12160, 1152, 2)


Validation loss decreased (0.01612 --> 0.01612).  Saving model ...
Epoch [1/100] Train Loss: 0.0134 Val Loss: 0.0161


EarlyStopping counter: 1 out of 10
the best of loss: 0.01612
Epoch [2/100] Train Loss: 0.0107 Val Loss: 0.0162


EarlyStopping counter: 2 out of 10
the best of loss: 0.01612
Epoch [3/100] Train Loss: 0.0099 Val Loss: 0.0169


Validation loss decreased (0.01612 --> 0.01589).  Saving model ...
Epoch [4/100] Train Loss: 0.0093 Val Loss: 0.0159


...

EarlyStopping counter: 6 out of 10
the best of loss: 0.01510
Epoch [12/100] Train Loss: 0.0074 Val Loss: 0.0175


EarlyStopping counter: 7 out of 10
the best of loss: 0.01510
Epoch [13/100] Train Loss: 0.0073 Val Loss: 0.0163


EarlyStopping counter: 8 out of 10
the best of loss: 0.01510
Epoch [14/100] Train Loss: 0.0072 Val Loss: 0.0169


EarlyStopping counter: 9 out of 10
the best of loss: 0.01510
Epoch [15/100] Train Loss: 0.0071 Val Loss: 0.0173


EarlyStopping counter: 10 out of 10
the best of loss: 0.01510
Early Stopping!


# test

In [35]:
config = {
    "learning_rate": 1e-3,
    "batch_size": 32,
    "epochs":100,
    "log_interval":100,
    "input_shape":[1,1152],
    "output_path": "outputs\\cnn\\0108",
    "patience":10    
          }

In [36]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_ckpt = ".\\"+config["output_path"]+"\\best.pth"
model = CNN(config, "not").to(device)
model.load_state_dict(torch.load(best_ckpt))

<All keys matched successfully>

In [57]:
test_dataset = PPG2ABPDataset_v3_Test(data_len=-1,data_root=r"..\data\processed\BP_npy\250107_1152\p00")
test_loader = DataLoader(test_dataset, batch_size=500, shuffle=False)

True
data prepared: (21696, 1152, 2)


In [58]:
output  = []
gt = []

for data in test_loader:
    _gt = data["gt_image"]
    cond = data["cond_image"]
    cond = cond.to(device)
    output.append(model(cond).detach().cpu().numpy())
    gt.append(_gt.detach().cpu().numpy())

In [59]:
output = np.concatenate(output)
gt = np.concatenate(gt)

In [67]:
output.shape,gt.shape

((21696, 1), (21696, 2))

In [61]:
scales = np.load(r"../data\processed\BP_npy\250107_1152\p00\scale_train.npy")
gt[:] -= scales[0,0]
gt[:] /= scales[0,1]
output[:] -= scales[0,0]
output[:] /= scales[0,1]

In [66]:
errors.shape

(21696, 2)

In [68]:
errors = output-gt[:,0]
me = np.mean(errors)
mae = np.mean(np.abs(errors))
rmse = np.sqrt(np.mean(errors**2))
std = np.std(errors)
print(me.shape,mae.shape,rmse.shape,std.shape)
n_samples = errors.shape[0]

error_5 = np.count_nonzero(np.abs(errors)<=5)/n_samples*100
error_10 = np.count_nonzero(np.abs(errors)<=10)/n_samples*100
error_15 = np.count_nonzero(np.abs(errors)<=15)/n_samples*100
# me = np.mean(errors,axis=0)
# mae = np.mean(np.abs(errors),axis=0)
# rmse = np.sqrt(np.mean(errors**2,axis=0))
# std = np.std(errors,axis=0)
# print(me.shape,mae.shape,rmse.shape,std.shape)
# n_samples = errors.shape[0]

# error_5 = np.count_nonzero(np.abs(errors)<=5,axis=0)/n_samples*100
# error_15 = np.count_nonzero(np.abs(errors)<=15,axis=0)/n_samples*100
# error_10 = np.count_nonzero(np.abs(errors)<=10,axis=0)/n_samples*100

() () () ()


In [65]:
mae

37.727077

In [69]:
print("""
          test data samples:
          # samples : {}
          
          Eval Stats:   DBP    SBP
          MAE:        {:6.3f} -
          RMSE:       {:6.3f} -
          Mean Error: {:6.3f} -
          STD:        {:6.3f} -
          
          BHS standards range:
          Error   <5mmHg <10mmHg <15mmHg
          gradeA     60%     85%     95%
          gradeB     50%     75%     90%
          gradeC     40%     65%     85%
          DBP     {:5.1f}%  {:5.1f}%  {:5.1f}%
           
          
          """.format(
            n_samples,
            mae,
            rmse,
            me,
            std,
            error_5, error_10, error_15,
            # error_5[1], error_10[1], error_15[1],
          ))


          test data samples:
          # samples : 21696
          
          Eval Stats:   DBP    SBP
          MAE:         8.070 -
          RMSE:       10.174 -
          Mean Error: -0.232 -
          STD:        10.171 -
          
          BHS standards range:
          Error   <5mmHg <10mmHg <15mmHg
          gradeA     60%     85%     95%
          gradeB     50%     75%     90%
          gradeC     40%     65%     85%
          DBP     797441.8%  1460944.2%  1917213.7%
           
          
          


In [31]:
print("""
          test data samples:
          # samples : {}
          
          Eval Stats:   DBP    SBP
          MAE:        {:6.3f} {:6.3f}
          RMSE:       {:6.3f} {:6.3f}
          Mean Error: {:6.3f} {:6.3f}
          STD:        {:6.3f} {:6.3f}
          
          BHS standards range:
          Error   <5mmHg <10mmHg <15mmHg
          gradeA     60%     85%     95%
          gradeB     50%     75%     90%
          gradeC     40%     65%     85%
          DBP     {:5.1f}%  {:5.1f}%  {:5.1f}%
          SBP     {:5.1f}%  {:5.1f}%  {:5.1f}%
           
          
          """.format(
            n_samples,
            *mae,
            *rmse,
            *me,
            *std,
            error_5[0], error_10[0], error_15[0],
            error_5[1], error_10[1], error_15[1],
          ))
print("## string for google spredsheet ##")
print(",".join(f"{x:.3f}" for x in[mae[0],rmse[0],me[0],std[0],mae[1],rmse[1],me[1],std[1]]))


          test data samples:
          # samples : 21696
          
          Eval Stats:   DBP    SBP
          MAE:         7.450 16.194
          RMSE:        9.661 19.907
          Mean Error: -1.356  2.898
          STD:         9.565 19.695
          
          BHS standards range:
          Error   <5mmHg <10mmHg <15mmHg
          gradeA     60%     85%     95%
          gradeB     50%     75%     90%
          gradeC     40%     65%     85%
          DBP      41.7%   73.7%   89.4%
          SBP      19.0%   36.0%   51.8%
           
          
          
## string for google spredsheet ##
7.450,9.661,-1.356,9.565,16.194,19.907,2.898,19.695
